### 调用百度API

In [1]:
from aip import AipNlp

APP_ID = '************'
API_KEY = '*********************'
SECRET_KEY = '****************************'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

### 读取训练集数据

In [81]:
count = 0
alldata = []
with open("./all_docs.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        alldata.append(line)
        count += 1 
print(count)
print(len(alldata))

108295
108295


In [82]:
train_id_list = []
train_title_content_list = []
train_dict = {}  # key = id, value = title+"。"+content
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        ids = line.split("\001")[0]
        title_content = line.split("\001")[1] + "。" + line.split("\001")[2]
        
        train_id_list.append(ids)
        train_title_content_list.append(title_content)
        
        train_dict[ids] = title_content

In [261]:
print(len(train_id_list))
print(len(train_title_content_list))
print(len(train_dict))

108295
108295
108295


### 读取测试集数据

In [84]:
'''
    获取测试集的数据id
'''
test_dict = {} # key = id, value = key words
test_id_list = [] # 只获取test data的id
test_keywords_list = []
with open("./train_docs_keywords.txt", "r", encoding="utf-8") as data:
    for line in data:
        line = line.strip()
        test_id_list.append(line.split("\t")[0])
        test_keywords_list.append(line.split("\t")[1])
        test_dict[line.split("\t")[0]] = line.split("\t")[1]

In [85]:
print(len(test_id_list))
print(len(test_keywords_list))
print(len(test_dict))

1000
1000
1000


### 获取测试集文本

In [86]:
test_data = []
for i in test_id_list:
    for k, v in train_dict.items():
        if i == k:
            test_data.append(v)

In [588]:
len(test_data)

1000

In [594]:
with open("./testData.txt", "w", encoding="utf-8") as test:
    for a,b,c in zip(test_id_list, test_data, test_keywords_list):
        test.write(str(a) + " " + str(b) + " " + str(c) + "\n")

In [245]:
# test_data[1]

### 利用百度分词

In [131]:
# 去除所有半角全角符号，只留字母、数字、中文。
def remove_punctuation(line):
    rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
    line = rule.sub('',line)
    return line

In [136]:
'''
    result = client.lexerCustom(''.join(e for e in line if e.isalnum()))不能去除所有的特殊符号
    result = client.lexerCustom(remove_punctuation(line)) 只保留中文数字字母
'''
import re
test_results = []
for num, line in enumerate(test_data):
    try:     
        result = client.lexerCustom(remove_punctuation(line))  
        cut_words = [item.get("item") for item in result.get("items")]
        test_results.append(" ".join(cut_words))
    except Exception as e:
        print(num + 1)
        print(e)

In [138]:
len(test_results)

1000

In [244]:
# test_results[1]

In [189]:
len(test_results[0].split(" "))

402

### 自定tfidf

In [210]:
import math
def train_idf(doc_list):
    idf_dic = {}
    tt_count = len(doc_list) # 获取整个文档集的长度
    
    # 在整个文档集中，每个单词出现的次数
    for doc in doc_list:
        for word in set(doc.split(" ")):
            idf_dic[word] = idf_dic.get(word, 0.0) + 1.0
    
    # 按公式计算idf值，分母加1平滑处理
    for k,v in idf_dic.items():
        idf_dic[k] = math.log(tt_count / (v+1.0))
    
    # 没在字典中的字，默认仅在一个文档中出现
    default_idf = math.log(tt_count / 1.0)
    
    return idf_dic, default_idf

In [211]:
# 统计tf值
def train_tf(word_list):
    tf_dic = {}
    for word in word_list.split(" "):
        tf_dic[word] = tf_dic.get(word, 0.0) + 1.0
    tt_count = len(word_list.split(" "))
    for k,v in tf_dic.items():
        tf_dic[k] = float(v) / tt_count
    return tf_dic

In [462]:
# 按公式计算tf-idf

# idf_dic, default_idf = train_idf(doc_list)

def get_tfidf(word_list):
    
    from functools import reduce
    
    tf_dic = train_tf(word_list)
    
    keyword = []
    tfidf_dic = {}
    
    for word in word_list.split(" "):
        
        idf = idf_dic.get(word, default_idf)
        tf = tf_dic.get(word, 0)
        
        tfidf = tf * idf
        tfidf_dic[word] = tfidf

    for k,v in sorted(tfidf_dic.items(), key=lambda x : x[1], reverse=True)[:10]:
        keyword.append(k)
        
    return reduce(lambda x,y:x+" "+y ,keyword)

    print()

### 对1000个测试集在百度的分词结果上用tfidf筛选前若干个关键词

In [220]:
'''
    获取测试集每篇的关键词个数
'''
true_len_for_each_id = []
for i in test_keywords_list:
    true_len_for_each_id.append(len(i.split(",")))

In [250]:
keywords = []
for i in test_results:
    keywords.append(get_tfidf(i))

In [255]:
'''
    按实际关键词的长度截取tfidf的结果
'''
common_part = []
for keyword, number in zip(keywords, true_len_for_each_id):
    common_part.append(",".join(keyword.split(" ")[:number]))

In [260]:
'''
    test_data中总共有2992个keywords
'''
from functools import reduce
reduce(lambda x,y : x+y, true_len_for_each_id)

2992

In [258]:
'''
    预测准确的关键词共1145个
'''
true_keywords_num = 0
for true, pred in zip(test_keywords_list, common_part):
    true_keywords_num += len([keyword for keyword in true.split(",") if keyword in pred.split(",")]) 
print(true_keywords_num)

1145


### 第二版答案 百度分词，实现tfidf,剔除长度为1的独字

### 对训练集分词

In [263]:
'''
    QPS会有限制
'''
import re
train_results = []
for num, line in enumerate(train_title_content_list):
    try:     
        result = client.lexerCustom(remove_punctuation(line))  
        cut_words = [item.get("item") for item in result.get("items")]
        train_results.append(" ".join(cut_words))
    except Exception as e:
        print(num + 1)
        print(e)

1529
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
4228
'NoneType' object is not iterable
11822
'NoneType' object is not iterable
14289
'NoneType' object is not iterable
20399
'NoneType' object is not iterable
37682
'NoneType' object is not iterable
38171
'NoneType' object is not iterable
39866
HTTPSConnectionPool(host='aip.baidubce.com', port=443): Max retries exceeded with url: /rpc/2.0/nlp/v1/lexer_custom?access_token=24.ecfd82c60fa16ebdadb9f2b88dd8e85b.2592000.1539316035.282335-11513666 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f0a4d30a588>: Failed to establish a new connection: [Errno -2] Name or service not known',))
39871
HTTPSConnectionPool(host='aip.baidubce.com', port=443): Max retries exceeded with url: /rpc/2.0/nlp/v1/lexer_custom?access_token=24.ecfd82c60fa16ebdadb9f2b88dd8e85b.2592000.1539316035.282335-11513666 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConne

In [265]:
108295 - 108233

62

In [294]:
copy_train_results = train_results # origial 

In [295]:
len(copy_train_results)

108233

In [325]:
copy_train_results_1 = train_results # 更正1个了

In [326]:
len(copy_train_results_1)

108234

In [406]:
copy_train_results_3 = train_results # 更正3个了
print(len(copy_train_results_3))

108236


### 批量处理因为文本过大造成的没有分词的文章

In [426]:
len(train_results)

108237

In [424]:
number.insert(0, 20398)

In [425]:
print(len(number))
print(number)

58
[20398, 37681, 38170, 39865, 39870, 44541, 49202, 52607, 53704, 55000, 56123, 56591, 61980, 62152, 64084, 64219, 64449, 64483, 64796, 67752, 69596, 69659, 70055, 71609, 71975, 72056, 72167, 72253, 72259, 72260, 72563, 75475, 78410, 83405, 88755, 89721, 90103, 90281, 90610, 90912, 90953, 92232, 92332, 92634, 92644, 92729, 93020, 93320, 93488, 93495, 94172, 95087, 96830, 96963, 98084, 99598, 103485, 107195]


In [428]:
for i in number:
    # 对每篇文章，在句号分词之后，每句话分词
    print("对第%d篇文章处理中..."%i)
    add_cut_words = []
    for line in train_title_content_list[int(i)].split("。"):
        sent = remove_punctuation(line)
        add_result = client.lexerCustom(sent) 
        try:
            add_cut_words.extend([item.get("item") for item in add_result.get("items")])
        except Exception as e:
            print(e)
            
    # 验证是否分词正确
    # 获取分词之后的所有字数
    total_len = 0
    for j in add_cut_words:
        total_len += len(j)
    print("文章包含正常符号数", total_len)
    # 获取分词之前去掉所有符号之后的长度

    if len(remove_punctuation(train_title_content_list[int(i)])) == total_len:
        print("分词前后文本的长度一致")
    
    # 将分词的格式转换
    add_cut_words = " ".join(add_cut_words)
    
    # 插入到应该插入的位置
    train_results.insert(int(i) ,add_cut_words)
    print("已插入对应位置")

对第20398篇文章处理中...
文章包含正常符号数 449
分词前后文本的长度一致
已插入对应位置
对第37681篇文章处理中...
文章包含正常符号数 14420
分词前后文本的长度一致
已插入对应位置
对第38170篇文章处理中...
文章包含正常符号数 12171
分词前后文本的长度一致
已插入对应位置
对第39865篇文章处理中...
文章包含正常符号数 1917
分词前后文本的长度一致
已插入对应位置
对第39870篇文章处理中...
'NoneType' object is not iterable
文章包含正常符号数 324
分词前后文本的长度一致
已插入对应位置
对第44541篇文章处理中...
文章包含正常符号数 23153
分词前后文本的长度一致
已插入对应位置
对第49202篇文章处理中...
'NoneType' object is not iterable
文章包含正常符号数 17063
分词前后文本的长度一致
已插入对应位置
对第52607篇文章处理中...
文章包含正常符号数 11402
分词前后文本的长度一致
已插入对应位置
对第53704篇文章处理中...
'NoneType' object is not iterable
文章包含正常符号数 11768
分词前后文本的长度一致
已插入对应位置
对第55000篇文章处理中...
'NoneType' object is not iterable
文章包含正常符号数 10431
分词前后文本的长度一致
已插入对应位置
对第56123篇文章处理中...
'NoneType' object is not iterable
文章包含正常符号数 11142
分词前后文本的长度一致
已插入对应位置
对第56591篇文章处理中...
文章包含正常符号数 12710
分词前后文本的长度一致
已插入对应位置
对第61980篇文章处理中...
'NoneType' object is not iterable
'NoneType' object is not iterable
文章包含正常符号数 11416
分词前后文本的长度一致
已插入对应位置
对第62152篇文章处理中...
'NoneType' object is not iterable
文章包含正常符号数 13561
分词前后文本的长度一致

### 统计每篇文章的tfidf，找出关键词

In [460]:
len(train_results)

108295

In [461]:
idf_dic, default_idf = train_idf(train_results)

In [465]:
keywords = []
for i in train_results:
    keywords.append(get_tfidf(i))

In [485]:
len(keywords)

108295

In [486]:
len(train_id_list)

108295

In [527]:
keywords[8]

'杨幂 4月29日 大幂幂 瘦 主编 并发文 托腮 张宇 漂亮 夸奖'

In [551]:
'''
    从10个关键词中提取两个关键词
'''
sents = []
for line in keywords:
    sent = []
    for word in line.split(" "):
        if len(word) != 1:
            sent.append(word)
            if len(sent) == 2:
                break
            sents.append(sent)

In [552]:
sent

['放入', '排骨']

In [574]:
",".join(sents[-5])

'花椒,姜蒜'

In [556]:
keywords_results = [",".join(sent) for sent in sents]

In [558]:
keywords_results[0]

'陈若仪,睫毛'

In [559]:
len(keywords_results)

108295

In [560]:
with open("./example2.csv", "w", encoding="utf-8") as data:
    data.write("id" + "," + "label1" + "," + "label2" + "\n")
    for i,j in zip(train_id_list, keywords_results):
        data.write(i + "," + j + "\n")

### 第三版答案，增加title的权重，在现有分词结果的基础上，将title再次分词，重复5遍，加入到原有的分词结果中

In [575]:
keywords[0]

'陈若仪 睫毛 面膜 林志颖 安利 洁面 日常 她 呐 隔离日'

In [577]:
'''
    只读取训练集的title
'''
train_title_list = []
for line in alldata:
    line = line.strip()
    if len(line.split("\001")) == 3:
        title = line.split("\001")[1]
        train_title_list.append(title)
print(len(train_title_list))

108295


In [587]:
'''
    对title进行分词
'''
import time
add_titles = []
count = 0
for num, title in enumerate(train_title_list):
    title = title.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk')
    add_result = client.lexerCustom(title) 
    try:
        add_titles.append([item.get("item") for item in add_result.get("items")])
        count += 1
    except Exception as e:
        print(num)
        print(e)  
        print("******************")
        count += 1

2700
'NoneType' object is not iterable
******************
2717
'NoneType' object is not iterable
******************
2726
'NoneType' object is not iterable
******************
2727
'NoneType' object is not iterable
******************
2736
'NoneType' object is not iterable
******************
2753
'NoneType' object is not iterable
******************
2758
'NoneType' object is not iterable
******************
2763
'NoneType' object is not iterable
******************
2764
'NoneType' object is not iterable
******************
2769
'NoneType' object is not iterable
******************
2774
'NoneType' object is not iterable
******************


KeyboardInterrupt: 

In [ ]:
'''
    分词的结果重复五倍
'''
